In [1]:
import os
import pandas as pd
import sleap
import numpy as np
import random
import re
import matplotlib.pyplot as plt

from pathlib import Path

In [2]:
# labels path
labels_path = "D:/SLEAP/20250102_generalizability_experiment/primary/canola/labels_canola_primary_6nodes.v005.slp" # actually canola and pennycress

# Directory output path to save the labels of the first crop
output_path = "D:/SLEAP/20250102_generalizability_experiment/primary/canola"

# Directory output path to save the labels of the second crop
output_path2 = "D:/SLEAP/20250102_generalizability_experiment/primary/pennycress"

In [3]:
# 'D:/SLEAP/SLEAP_Canola_Pennycress/primary_root/h5_files_for_PR_sleap_project/9_do/33PFABDMTW.h5'
# Z:/Experiments/Canola/CYL_Canola_Diversity_Screen/Downstream_Data_Extraction_and_Analysis/4-19-23_data_v000_not_qc_not_proofread/H5_w_SLEAP_predictions_preqc_preproofread/W22/2do/HMDQXXWRSQ.h5
# D:\SLEAP\SLEAP_Canola_Pennycress\primary_root\h5_files_for_PR_sleap_project\pennycress\14_do

def separate_crops(labels, match_string='pennycress'):
    """Load sleap labels and separate the crops using matches from the video filenames.
    
    Args:
        labels (sleap.Labels): Labels to separate.
        
    Returns:
        first_crop_labels (sleap.Labels): Labels using the first crop.
        second_crop_labels (sleap.Labels): Labels using the second crop.
    """
    first_crop_labels = []
    second_crop_labels = []
    for label in labels:
        if match_string in label.video.filename:
            second_crop_labels.append(label)
            print(f"Found {match_string} data: {label.video.filename}")
        else:
            first_crop_labels.append(label)
    first_crop_labels = sleap.Labels(first_crop_labels)
    second_crop_labels = sleap.Labels(second_crop_labels)
    print(f"Number of first crop labels: {len(first_crop_labels)}")
    print(f"Number of second crop labels: {len(second_crop_labels)}")
    return first_crop_labels, second_crop_labels

In [4]:
base_labels = sleap.load_file(Path(labels_path).as_posix()) # load labels file
user_labels = base_labels.with_user_labels_only() # user instances only

In [5]:
# Make the output directory if it doesn't exist
Path(output_path).mkdir(parents=True, exist_ok=True)

In [6]:
# Print the first few items to check their structure
for label in user_labels[:5]:
    print(label)

LabeledFrame(video=HDF5Video('D:/SLEAP/SLEAP_Canola_Pennycress/primary_root/h5_files_for_PR_sleap_project/9_do/33PFABDMTW.h5'), frame_idx=0, instances=1)
LabeledFrame(video=HDF5Video('D:/SLEAP/SLEAP_Canola_Pennycress/primary_root/h5_files_for_PR_sleap_project/9_do/33PFABDMTW.h5'), frame_idx=27, instances=1)
LabeledFrame(video=HDF5Video('D:/SLEAP/SLEAP_Canola_Pennycress/primary_root/h5_files_for_PR_sleap_project/9_do/CTDUW5MAB7.h5'), frame_idx=14, instances=1)
LabeledFrame(video=HDF5Video('D:/SLEAP/SLEAP_Canola_Pennycress/primary_root/h5_files_for_PR_sleap_project/9_do/CTDUW5MAB7.h5'), frame_idx=59, instances=1)
LabeledFrame(video=HDF5Video('D:/SLEAP/SLEAP_Canola_Pennycress/primary_root/h5_files_for_PR_sleap_project/9_do/8SA030IT8T.h5'), frame_idx=49, instances=1)


In [7]:
print(f"There are {len(user_labels)} labels total.")

# Separate the pennycress and canola data
canola_labels, pennycress_labels = separate_crops(user_labels)

# Set pennycress labels output_path
pennycress_output_path = Path("D:/SLEAP/20250102_generalizability_experiment/primary") / "pennycress"
Path(pennycress_output_path).mkdir(parents=True, exist_ok=True)

# Save the pennycress labels
pennycress_labels.save(pennycress_output_path / "labels_pennycress_primary_6nodes.v000.slp")
print(f"Saved pennycress labels to {pennycress_output_path.as_posix()}")

# Save the canola labels
canola_labels.save(Path(output_path) / "labels_ONLYcanola_primary_6nodes.v000.slp")
print(f"Saved canola labels to {output_path}")

There are 435 labels total.
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Canola_Pennycress/lateral_3_nodes/h5_files_for_LR_sleap_project/pennycress/14_do/C4SII6QM92.h5
Found pennycress data: D:/SLEAP/SLEAP_Can